CALL BACKS ARE USEFUL FOR VARIOUS PURPOSES
TO CHECK WHERE THE MODEL WENT WRONG
THE PROGRESS BAR OF THE MODEL
TO TEST THE MODEL
TO SEE WHICH IS THE BEST MODEL AFTER TRAINING WITH A LOT OF EPOCHS

Checkpoints will allow you to continue your training from where you left out

In [17]:
import os
os.chdir(r"\Users\Akhil\OneDrive\Desktop\animal-disease-classification-with-llm")

In [18]:
from dataclasses import dataclass   
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [19]:
from classifier.constants import *
from classifier.utils.common import read_yaml, create_directories

In [20]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath: Path = CONFIG_FILE_PATH,
                 params_filepath: Path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([Path(self.config.artifacts_root)])

    def get_prepare_callbacks_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories(
            [
                Path(model_ckpt_dir),
                Path(config.tensorboard_root_log_dir)
            ]
        )
        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )
        return prepare_callback_config


In [21]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [22]:
# You can get this from tensorflow documentation itself!

class PrepareCallbacks:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{time.strftime('%Y%m%d-%H%M%S')}"
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )
    
    def get_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [14]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callbacks_config()
    prepare_callbacks = PrepareCallbacks(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_callbacks()
except Exception as e:
    raise e

[2026-01-18 14:45:40,407 - INFO - common - yaml file: config\config.yaml loaded successfully]
[2026-01-18 14:45:40,410 - INFO - common - yaml file: params.yaml loaded successfully]
created directory at: artifacts
created directory at: artifacts\prepare_callbacks\checkpoint_dir
created directory at: artifacts\prepare_callbacks\tensorboard_log_dir


In [16]:
import time
timestamp = time.strftime("%Y%m%d-%H%M%S")
print(f"Current Timestamp: {timestamp}")

Current Timestamp: 20260118-144702
